In [1]:
from tqdm import tqdm

In [2]:
import re
sequence_length = 80
def clean_str(string):

    #print string
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", "", string)
    string = re.sub(r"\)", "", string)
    string = re.sub(r"\?", "", string)
    string = re.sub(r"/", "", string)
    string = re.sub(r"\s{2,}", " ", string)
    #print (string)
    return string.strip().lower()


def pad_sentences(sentences, padding_word="<PAD/>"):
    padded_sentences = []
    for i in range(len(sentences)):
        sentence = sentences[i]
        num_padding = sequence_length - len(sentence)
        #print (sentence)
        if num_padding<=0:
            temp=sentence[0:sequence_length-1]
            temp.append(padding_word)
            new_sentence=temp
        else:
            new_sentence = sentence + [padding_word] * num_padding
        
        padded_sentences.append(new_sentence)
    #print (padded_sentences[0],len(padded_sentences[0]))
    return padded_sentences




In [3]:
from keras.models import load_model
model = load_model('version7.h5')

Using TensorFlow backend.
/Users/atalmanuja/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [4]:
import pickle
import numpy as np
pickle_in = open("vocab.pickle","rb")
vocabulary = pickle.load(pickle_in)

In [5]:
embedding_weights = np.load('embedding_weights.npy')

In [6]:
##questions framework

In [19]:
questions = ['You find it difficult to introduce yourself to other people',
            'You are usually highly motivated and energetic',
            'Your mood can change very quickly',
            'You rarely worry about how your actions affect other people',
            'In a discussion, truth should be more important than people s sensitivities',
            'You do not let other people influence your actions',
            'When you sleep, your dreams tend to focus on the real world and its events',
            'Your emotions control you more than you control them',
            'You are more of a natural improviser than a careful planner',
            'You are a relatively reserved and quiet person',
            'Logic is usually more important than heart when it comes to making important decisions',
            'If your friend is sad about something, you are more likely to offer emotional support than suggest ways to deal with the problem',
            'Being right is more important than being cooperative when it comes to teamwork',
            'You see yourself as very emotionally stable',
            'You usually find it difficult to relax when talking in front of many people',
            'You worry too much about what other people think',
            'Generally speaking, you rely more on your experience than your imagination',
            'You feel very anxious in stressful situations',
            'You often take initiative in social situations',
             'You believe that it is more rewarding to be liked by others than to be powerful',
             'You have always been interested in unconventional and ambiguous things, e.g. in books, art, or movies'
             ]

In [54]:
questions = ['You find it difficult to introduce yourself to other people',#d
            'You are usually highly motivated and energetic',#a
            'Your mood can change very quickly',#d
            'You rarely worry about how your actions affect other people',#a
            'In a discussion, truth should be more important than people s sensitivities',#a
            'You do not let other people influence your actions',#a
            'When you sleep, your dreams tend to focus on the real world and its events',#a
             ]

In [55]:
labels = []
for qn in questions_working:
    l = predict_custom(qn)
    labels.append(l)

In [56]:
freq = {
    'E':0,
    'I':0,
    'N':0,
    'S':0,
    'T':0,
    'F':0,
    'J':0,
    'P':0
}

In [57]:
for qn in questions_working:
    predicted = predict_custom(qn)
    if predicted[0] == 1:
        freq['E'] = freq['E'] + 1
    else:
        freq['I'] = freq['I'] + 1
    if predicted[1] == 1:
        freq['N'] = freq['N'] + 1
    else:
        freq['S'] = freq['S'] + 1
    if predicted[2] == 1:
        freq['T'] = freq['T'] + 1
    else:
        freq['F'] = freq['F'] + 1
    if predicted[3] == 1:
        freq['P'] = freq['P'] + 1
    else:
        freq['J'] = freq['J'] + 1    

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [58]:
def predict_custom(query):
    query = clean_str(query)
    query = query.split()
    x_=pad_sentences([query])
    x = []
    keys = list(vocabulary.keys())
    for sentence in x_:
        for word in sentence:
            if word in keys:
                x.append(vocabulary[word])
            else:
                x.append(0)
    x = np.array([x])
    x=embedding_weights[0][x]
    prediction = model.predict(x) 
    for i in range(4):
        if prediction[0][i]>0.5:
            prediction[0][i] = 1
        else:
            prediction[0][i] = 0
    return prediction         
    

In [59]:
predict_custom('hello what are you doing here')

array([[ 0.,  0.,  0.,  1.]], dtype=float32)

In [60]:
answers = []

In [61]:
for question in questions:
    answers.append(predict_custom(question))

In [62]:
answers

[array([[ 0.,  0.,  1.,  0.]], dtype=float32),
 array([[ 0.,  1.,  0.,  1.]], dtype=float32),
 array([[ 1.,  0.,  0.,  0.]], dtype=float32),
 array([[ 0.,  0.,  0.,  1.]], dtype=float32),
 array([[ 0.,  0.,  0.,  0.]], dtype=float32),
 array([[ 1.,  0.,  0.,  0.]], dtype=float32),
 array([[ 0.,  1.,  0.,  0.]], dtype=float32)]

In [63]:
x = np.load('x.npy')

In [64]:
y = np.load('y.npy')

In [65]:
len(y)

50000

In [66]:
from tqdm import tqdm

In [67]:
threshold = 0.70
correct = 0
for i in tqdm(range(len(x))):
    predict = model.predict(x[i].reshape(1,80,300))
    for i in range(4):
        if predict[0][i]>0.5:
            predict[0][i] = 1
        else:
            predict[0][i] = 0
    if comparing2arrays(predict,y[i]):
        correct = correct + 1
print(correct)        

  0%|          | 139/50000 [00:03<18:28, 44.99it/s]


KeyboardInterrupt: 

In [68]:
def comparing2arrays(ar1, ar2):
    for i in range(4):
        if ar1[0][i] != ar2[0]:
            return False
    return True    

In [69]:
y[0].shape

(4,)

In [70]:
personalities = {'ISFJ':[0,0,0,0],
                'ISFP':[0,0,0,1],
                'ISTJ':[0,0,1,0],
                'ISTP':[0,0,1,1],
                'INFJ':[0,1,0,0],
                'INFP':[0,1,0,1],
                'INTJ':[0,1,1,0],
                'INTP':[0,1,1,1],
                'ESFJ':[1,0,0,0],
                'ESFP':[1,0,0,1],
                'ESTJ':[1,0,1,0],
                'ESTP':[1,0,1,1],
                'ENFJ':[1,1,0,0],
                'ENFP':[1,1,0,1],
                'ENTJ':[1,1,1,0],
                'ENTP':[1,1,1,1]
                }

In [71]:
#script for teh teminal

In [72]:
print('you re going to start a personality test. Answer the questions truly as it would give accurate results')

you re going to start a personality test. Answer the questions truly as it would give accurate results


In [73]:
def ask():
    print('press a if you agree')
    print('press d if you disagree')
    responses = []
    for i in range(len(questions)):
        print(questions[i])
        r = input('press answer key')
        if r=='a':
            responses.append(answers[i])
        else:
            responses.append(compliment_answer(answers[i]))
    print('thanks!')        
    print('loading result.......')
    return responses
    

In [74]:
responses = ask()


press a if you agree
press d if you disagree
You find it difficult to introduce yourself to other people


KeyboardInterrupt: 

In [75]:
def compliment_answer(ans):
    for i in range(4):
        if ans[0][i]==1:
            ans[0][i]=0
        else:
            ans[0][i]=1
    return ans    
    

In [76]:
compliment_answer(responses[0])

NameError: name 'responses' is not defined

In [77]:
def evaluate(responses):
    freq = {
    'E':0,
    'I':0,
    'N':0,
    'S':0,
    'T':0,
    'F':0,
    'J':0,
    'P':0
    }
    for response in responses:
            if response[0][0] == 1:
                freq['E'] = freq['E'] + 1
            else:
                freq['I'] = freq['I'] + 1
            if response[0][1] == 1:
                freq['N'] = freq['N'] + 1
            else:
                freq['S'] = freq['S'] + 1
            if response[0][2] == 1:
                freq['T'] = freq['T'] + 1
            else:
                freq['F'] = freq['F'] + 1
            if response[0][3] == 1:
                freq['P'] = freq['P'] + 1
            else:
                freq['J'] = freq['J'] + 1    

    return freq

In [78]:
def evaluate_final(freq):
    personality = ''
    if freq['E'] >=freq['I']:
        personality = personality + 'E'
    else:
        personality = personality + 'I'
    if freq['N'] >=freq['S']:
        personality = personality+'N'
    else:
        personality = personality+'S'    
    if freq['T'] >=freq['F']:
        personality = personality+'T'
    else:
        personality = personality+'F'
    if freq['J'] >=freq['P']:
        personality = personality+'J'
    else:
        personality =personality+ 'P'    
        
    return personality

In [79]:
evaluate_final(evaluate(responses))

NameError: name 'responses' is not defined

In [80]:
def final():
    responses = ask()
    freq = evaluate(responses)
    personality = evaluate_final(freq)
    print('your personality is '+personality)
    print(personality_desc[personality])
    

In [50]:
final()

press a if you agree
press d if you disagree
You find it difficult to introduce yourself to other people
press answer keya
You are usually highly motivated and energetic
press answer keyd
Your mood can change very quickly
press answer keyd
You rarely worry about how your actions affect other people
press answer keyd
In a discussion, truth should be more important than people s sensitivities
press answer keyd
You do not let other people influence your actions
press answer keyd
When you sleep, your dreams tend to focus on the real world and its events
press answer keyd
Your emotions control you more than you control them
press answer key
You are more of a natural improviser than a careful planner
press answer keyd
You are a relatively reserved and quiet person
press answer keyd
Logic is usually more important than heart when it comes to making important decisions
press answer keyd
If your friend is sad about something, you are more likely to offer emotional support than suggest ways to d

In [48]:
personality_desc = {'ISFJ':'Very dedicated and warm protectors, always ready to defend their loved ones',
                'ISFP':'Flexible and charming artists, always ready to explore and experience something new',
                'ISTJ':'Practical and fact-minded individuals, whose reliability cannot be doubted',
                'ISTP':'Bold and practical experimenters, masters of all kinds of tools',
                'INFJ':'Quiet and mystical, yet very inspiring and tireless idealists',
                'INFP':'Poetic, kind and altruistic people, always eager to help a good cause',
                'INTJ':'Imaginative and strategic thinkers,with a plan for everything',
                'INTP':'Innovative inventors with an unquenchable thirst for knowledge',
                'ESFJ':'Very dedicated and warm protectors always ready to defend their loved ones',
                'ESFP':'Spontaneous, energetic and enthusiastic people ñ life is never boring around them',
                'ESTJ':'Excellent administrators, unsurpassed at managing things ñ or people',
                'ESTP':'Smart, energetic and very perceptive people, who truly enjoy living on the edge',
                'ENFJ':'Charismatic and inspiring leaders, able to mesmerize their listeners',
                'ENFP':'Enthusiastic, creative and sociable free spirits, who can always find a reason to smile',
                'ENTJ':'Bold,imaginative and strong-willed leaders,always finding a way -or making one',
                'ENTP':'Smart and curious thinkers who cannot resist an intellectual challenge'
                }

In [51]:
final()

press a if you agree
press d if you disagree
You find it difficult to introduce yourself to other people


KeyboardInterrupt: 

In [52]:
final()

press a if you agree
press d if you disagree
You find it difficult to introduce yourself to other people
press answer keya
You are usually highly motivated and energetic
press answer keys
Your mood can change very quickly
press answer keya
You rarely worry about how your actions affect other people
press answer keys
In a discussion, truth should be more important than people s sensitivities
press answer keya
You do not let other people influence your actions
press answer keys
When you sleep, your dreams tend to focus on the real world and its events
press answer keya
Your emotions control you more than you control them
press answer keys
You are more of a natural improviser than a careful planner
press answer keys
You are a relatively reserved and quiet person
press answer keys
Logic is usually more important than heart when it comes to making important decisions
press answer keyf
If your friend is sad about something, you are more likely to offer emotional support than suggest ways to 

In [84]:
final()

press a if you agree
press d if you disagree
You find it difficult to introduce yourself to other people
press answer keya
You are usually highly motivated and energetic
press answer keya
Your mood can change very quickly
press answer keyd
You rarely worry about how your actions affect other people
press answer keya
In a discussion, truth should be more important than people s sensitivities
press answer keya
You do not let other people influence your actions
press answer keya
When you sleep, your dreams tend to focus on the real world and its events
press answer keyd
thanks!
loading result.......
your personality is ISFP
Flexible and charming artists, always ready to explore and experience something new
